# <center> Email Spam Classification Dataset CSV

**In this project**
- **Applying classifier with hyperparameter tunnig**
    - performing Decision tree with hyperparameter tunnig using grid search
    - performing RandomForest with hyperparameter tunnig using grid search
    - performing SVM with hyperparameter tunnig using grid search
- **Applying Dimentional reduction and dropping unnecessary data**
    - examining the effect of dropping the least correlated features on the final results
    - Examinig the effect of PCA 
    - Applying Randomforest on new dataset

**Project information**
- This is a csv file containing related information of 5172 randomly picked email files and their respective labels for spam or  not-spam classification.
- **Link of dataset**  :  
https://www.kaggle.com/datasets/balaka18/email-spam-classification-dataset-csv

In [1]:
# import libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
#-------------------------Sklearn classifiers ----------------------------#
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC 
#-------------------
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler,StandardScaler
from sklearn.metrics import confusion_matrix, precision_score,recall_score,f1_score, accuracy_score


In [2]:
df=pd.read_csv('emails.csv')
print(df.shape)
df.head()

(5172, 3002)


,Email No.,the,to,ect,and,for,of,a,you,hou,...,connevey,jay,valued,lay,infrastructure,military,allowing,ff,dry,Prediction
0,Email 1,0,0,1,0,0,0,2,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Email 2,8,13,24,6,6,2,102,1,27,...,0,0,0,0,0,0,0,1,0,0
2,Email 3,0,0,1,0,0,0,8,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Email 4,0,5,22,0,5,1,51,2,10,...,0,0,0,0,0,0,0,0,0,0
4,Email 5,7,6,17,1,5,2,57,0,9,...,0,0,0,0,0,0,0,1,0,0


In [3]:
feature_names=list(df.columns)

In [4]:
for i in feature_names:
    if df[i].dtypes=='object':
        print(i,len(set(df[i])))
    

Email No. 5172


In [5]:
# checking for missing data
df.isnull().any().value_counts()

False    3002
dtype: int64

In [6]:
df=df.drop(['Email No.'],axis=1)
print(df.shape)
df.head()

(5172, 3001)


,the,to,ect,and,for,of,a,you,hou,in,...,connevey,jay,valued,lay,infrastructure,military,allowing,ff,dry,Prediction
0,0,0,1,0,0,0,2,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,8,13,24,6,6,2,102,1,27,18,...,0,0,0,0,0,0,0,1,0,0
2,0,0,1,0,0,0,8,0,0,4,...,0,0,0,0,0,0,0,0,0,0
3,0,5,22,0,5,1,51,2,10,1,...,0,0,0,0,0,0,0,0,0,0
4,7,6,17,1,5,2,57,0,9,3,...,0,0,0,0,0,0,0,1,0,0


In [7]:
# droping the least correlated columns
A=[]
for i in df.columns:
    A.append(abs(df[i].corr(df['Prediction'])))
A_zip = zip(df.columns, A)
corr=sorted(A_zip, key=lambda x: x[1])

In [8]:
def drop_least_corr(df,corr,a):
    A_droped=[]
    for i in range(a):
        A_droped.append(corr[0:a][i][0])
    df=df.drop(A_droped,axis=1)
    return A_droped,df

In [9]:
def drop_leastcorr(df,corr,a):
    A_droped=[]
    for i in range(a):
        A_droped.append(corr[0:a][i][0])
    df=df.drop(A_droped,axis=1)
    v=df.shape[1]-1
    X=df.iloc[:, :v]
    y=df.iloc[:, -1]
    #
    X_code, X_val, y_code, y_val = train_test_split(X,y, test_size = 0.1,stratify=y, random_state=40)
    X_train, X_test, y_train, y_test = train_test_split(X_code,y_code, test_size = 0.2,stratify=y_code,random_state=40)
    #
    scale=StandardScaler()
    X_train=scale.fit_transform(X_train)
    X_test=scale.transform(X_test)
    return X_train, X_test,X_val,y_train, y_test,y_val

In [10]:
#df=drop_least_corr(df,corr,25)[1]

In [11]:
#divide the dataset into features(X) and target (y)
v=df.shape[1]-1
X=df.iloc[:, :v]
y=df.iloc[:, -1]
X.shape, y.shape

((5172, 3000), (5172,))

### data would be devided into 3 parts 
- **training**
- **testing**
- **validation(at the end )**

In [12]:
from sklearn.model_selection import train_test_split
X_code, X_val, y_code, y_val = train_test_split(X,y, test_size = 0.1,stratify=y, random_state=40)
X_train, X_test, y_train, y_test = train_test_split(X_code,y_code, test_size = 0.2,stratify=y_code,random_state=40)
print(df.shape,X_train.shape, X_test.shape, X_val.shape)

(5172, 3001) (3723, 3000) (931, 3000) (518, 3000)


In [13]:
# Scalling data
from sklearn.preprocessing import MinMaxScaler,StandardScaler
#scale=MinMaxScaler()
scale=StandardScaler()
X_train=scale.fit_transform(X_train)
X_test=scale.transform(X_test)
#print(X_train.min(),X_train.max())
#print(X_test.min(),X_test.max())

In [14]:
X_train, X_test,X_val,y_train, y_test,y_val=drop_leastcorr(df,corr,10)

## Decision tree

In [15]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV
clf =DecisionTreeClassifier()
print(clf.get_params())
param_grid = {
    'max_depth': [20, 25 , 30],
    'min_samples_leaf': [1,5],
    'max_features': [None,0.1, 0.3]
     }
# Instantiate a 10-fold CV grid search object 'grid_dt'
grid_dt= GridSearchCV(estimator=clf,
                       param_grid= param_grid,
                       scoring='accuracy',
                       cv=10,
                       n_jobs=8)

# Fit 'grid_dt' to the training data
grid_dt.fit(X_train, y_train)

# Extract best hyperparameters from 'grid_dt' and print them
best_hyperparams = grid_dt.best_params_
best_CV_score = grid_dt.best_score_
print(best_hyperparams,best_CV_score)

{'ccp_alpha': 0.0, 'class_weight': None, 'criterion': 'gini', 'max_depth': None, 'max_features': None, 'max_leaf_nodes': None, 'min_impurity_decrease': 0.0, 'min_impurity_split': None, 'min_samples_leaf': 1, 'min_samples_split': 2, 'min_weight_fraction_leaf': 0.0, 'random_state': None, 'splitter': 'best'}
{'max_depth': 30, 'max_features': None, 'min_samples_leaf': 1} 0.9285623684741561


In [21]:
from sklearn.tree import DecisionTreeClassifier
#instantiate object
clf =DecisionTreeClassifier(
    max_depth = 30,
    max_features=None,
    min_samples_leaf=1,
    random_state = 1
)
#
#clf =DecisionTreeClassifier(
#    max_depth = 30,
#    random_state = 1
#)
#
clf.fit(X_train,y_train)

DecisionTreeClassifier(max_depth=30, random_state=1)

In [22]:
#training 
y_pred_train=clf.predict(X_train)
#------------------------------------------#
#testing
y_pred_test=clf.predict(X_test)

In [23]:
from sklearn.metrics import confusion_matrix, precision_score,recall_score,f1_score, accuracy_score
cm = confusion_matrix(y_test,y_pred_test)
f1 = f1_score(y_test,y_pred_test)
acc= accuracy_score(y_test,y_pred_test)
cm,f1,acc

(array([[636,  25],
        [ 38, 232]], dtype=int64),
 0.8804554079696395,
 0.9323308270676691)

## Random Forest

In [19]:
from sklearn.ensemble import RandomForestClassifier
clf =RandomForestClassifier()
print(clf.get_params())
param_grid = {
    'n_estimators': [10,50,100,150,200]
     }
# Instantiate a 10-fold CV grid search object 'grid_dt'
grid_dt= GridSearchCV(estimator=clf,
                       param_grid= param_grid,
                       scoring='accuracy',
                       cv=10,
                       n_jobs=8)

# Fit 'grid_dt' to the training data
grid_dt.fit(X_train, y_train)

# Extract best hyperparameters from 'grid_dt' and print them
best_hyperparams = grid_dt.best_params_
best_CV_score = grid_dt.best_score_
print(best_hyperparams,best_CV_score)


{'bootstrap': True, 'ccp_alpha': 0.0, 'class_weight': None, 'criterion': 'gini', 'max_depth': None, 'max_features': 'auto', 'max_leaf_nodes': None, 'max_samples': None, 'min_impurity_decrease': 0.0, 'min_impurity_split': None, 'min_samples_leaf': 1, 'min_samples_split': 2, 'min_weight_fraction_leaf': 0.0, 'n_estimators': 100, 'n_jobs': None, 'oob_score': False, 'random_state': None, 'verbose': 0, 'warm_start': False}
{'n_estimators': 200} 0.972071117645363


In [24]:
#
clf = RandomForestClassifier(n_estimators=200).fit(X_train, y_train)
#
#training 
y_pred_train=clf.predict(X_train)
#------------------------------------------#
#testing
y_pred_test=clf.predict(X_test)
#
cm = confusion_matrix(y_test,y_pred_test)
f1 = f1_score(y_test,y_pred_test)
acc= accuracy_score(y_test,y_pred_test)
cm,f1,acc

(array([[652,   9],
        [ 18, 252]], dtype=int64),
 0.9491525423728815,
 0.9709989258861439)

## SVM

In [32]:
X_train, X_test,X_val,y_train, y_test,y_val=drop_leastcorr(df,corr,0)

In [33]:
from sklearn.svm import SVC
clf=SVC()
print(clf.get_params)
param_grid = [{'kernel': ['linear','poly','sigmoid']}]
# Instantiate a 10-fold CV grid search object 'grid_dt'
grid_dt= GridSearchCV(estimator=clf,
                       param_grid= param_grid,
                       scoring='accuracy',
                       cv=10,
                       n_jobs=8)

# Fit 'grid_dt' to the training data
grid_dt.fit(X_train, y_train)

# Extract best hyperparameters from 'grid_dt' and print them
best_hyperparams = grid_dt.best_params_
best_CV_score = grid_dt.best_score_
print(best_hyperparams,best_CV_score)

<bound method BaseEstimator.get_params of SVC()>
{'kernel': 'linear'} 0.9425214044798063


In [34]:
#
clf = SVC(kernel= 'linear').fit(X_train, y_train)
#
#training 
y_pred_train=clf.predict(X_train)
#------------------------------------------#
#testing
y_pred_test=clf.predict(X_test)
#
cm = confusion_matrix(y_test,y_pred_test)
f1 = f1_score(y_test,y_pred_test)
acc= accuracy_score(y_test,y_pred_test)
cm,f1,acc

(array([[632,  29],
        [ 24, 246]], dtype=int64),
 0.9027522935779817,
 0.9430719656283566)

# <center> Classification using dimentional reduction

In [47]:
#Removing the lowest 24 correlated features
corr[10:24]
df=drop_least_corr(df,corr,25)[1]
df.head()

,the,to,ect,and,for,of,a,you,hou,in,...,connevey,jay,valued,lay,infrastructure,military,allowing,ff,dry,Prediction
0,0,0,1,0,0,0,2,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,8,13,24,6,6,2,102,1,27,18,...,0,0,0,0,0,0,0,1,0,0
2,0,0,1,0,0,0,8,0,0,4,...,0,0,0,0,0,0,0,0,0,0
3,0,5,22,0,5,1,51,2,10,1,...,0,0,0,0,0,0,0,0,0,0
4,7,6,17,1,5,2,57,0,9,3,...,0,0,0,0,0,0,0,1,0,0


In [56]:
df_feature=df.drop(['Prediction'],axis=1)
df_feature

,the,to,ect,and,for,of,a,you,hou,in,...,enhancements,connevey,jay,valued,lay,infrastructure,military,allowing,ff,dry
0,0,0,1,0,0,0,2,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,8,13,24,6,6,2,102,1,27,18,...,0,0,0,0,0,0,0,0,1,0
2,0,0,1,0,0,0,8,0,0,4,...,0,0,0,0,0,0,0,0,0,0
3,0,5,22,0,5,1,51,2,10,1,...,0,0,0,0,0,0,0,0,0,0
4,7,6,17,1,5,2,57,0,9,3,...,0,0,0,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5167,2,2,2,3,0,0,32,0,0,5,...,0,0,0,0,0,0,0,0,0,0
5168,35,27,11,2,6,5,151,4,3,23,...,0,0,0,0,0,0,0,0,1,0
5169,0,0,1,1,0,0,11,0,0,1,...,0,0,0,0,0,0,0,0,0,0
5170,2,7,1,0,2,1,28,2,0,8,...,0,0,0,0,0,0,0,0,1,0


In [92]:
from sklearn.decomposition import PCA
pca=PCA(n_components=10).fit(df_feature.values)
print(pca.explained_variance_ratio_ )
new_df_pca=pd.DataFrame(pca.transform(df_feature.values),columns=['pc1','pc2','pc3','pc4','pc5','pc6','pc7','pc8','pc9','pc10'])
new_df_pca

[0.92817932 0.01342913 0.01085934 0.00662267 0.00486287 0.00394223
 0.00263508 0.0023259  0.00205327 0.00193223]


,pc1,pc2,pc3,pc4,pc5,pc6,pc7,pc8,pc9,pc10
0,-171.412156,-0.505930,2.294228,-0.193560,3.410386,3.680480,-5.306440,-2.605819,4.291722,0.342619
1,146.994893,32.322789,-15.841536,-9.618026,19.590001,3.410485,26.148679,-32.013173,-6.720390,-8.647134
2,-164.408868,-0.901846,8.223857,-0.229763,4.552285,5.706720,-6.702110,6.247881,1.854008,-4.148842
3,-27.212277,36.211985,-25.167273,-3.484085,-4.221186,-1.915129,-4.069590,-20.336791,0.731665,-2.775252
4,-17.523877,25.220073,-17.457292,6.718774,0.995999,-0.602596,-3.230900,3.105801,3.604785,-15.709520
...,...,...,...,...,...,...,...,...,...,...
5167,-102.205568,-1.681549,4.404483,-3.672530,1.475927,6.139174,-8.147510,9.630431,9.517563,-1.421933
5168,274.153159,-16.752035,-38.865978,18.862895,-20.546264,-34.017309,-1.690690,9.324336,7.061650,3.759435
5169,-151.132048,-3.140374,6.314800,0.689275,4.691984,0.985457,-7.851108,-0.151008,2.594627,0.538779
5170,-61.047578,-11.620199,5.793348,10.999819,13.416779,9.854499,1.442415,-14.112538,-5.393635,-4.921217


In [93]:
new_df_pca['Prediction']=df['Prediction']
print(new_df_pca.shape)
new_df_pca.head()

(5172, 11)


,pc1,pc2,pc3,pc4,pc5,pc6,pc7,pc8,pc9,pc10,Prediction
0,-171.412156,-0.505930,2.294228,-0.193560,3.410386,3.680480,-5.306440,-2.605819,4.291722,0.342619,0
1,146.994893,32.322789,-15.841536,-9.618026,19.590001,3.410485,26.148679,-32.013173,-6.720390,-8.647134,0
2,-164.408868,-0.901846,8.223857,-0.229763,4.552285,5.706720,-6.702110,6.247881,1.854008,-4.148842,0
3,-27.212277,36.211985,-25.167273,-3.484085,-4.221186,-1.915129,-4.069590,-20.336791,0.731665,-2.775252,0
4,-17.523877,25.220073,-17.457292,6.718774,0.995999,-0.602596,-3.230900,3.105801,3.604785,-15.709520,0


In [94]:
#divide the dataset into features(X) and target (y)
v=new_df_pca.shape[1]-1
X=new_df_pca.iloc[:, :v].values
y=new_df_pca.iloc[:, -1].values
X.shape, y.shape

((5172, 10), (5172,))

In [95]:
from sklearn.model_selection import train_test_split
X_code, X_val, y_code, y_val = train_test_split(X,y, test_size = 0.1,stratify=y, random_state=40)
X_train, X_test, y_train, y_test = train_test_split(X_code,y_code, test_size = 0.2,stratify=y_code,random_state=40)
print(df.shape,X_train.shape, X_test.shape, X_val.shape)

(5172, 2976) (3723, 10) (931, 10) (518, 10)


In [96]:
# Scalling data
from sklearn.preprocessing import MinMaxScaler,StandardScaler
scale=MinMaxScaler()
#scale=StandardScaler()
X_train=scale.fit_transform(X_train)
X_test=scale.transform(X_test)
print(X_train.min(),X_train.max())
print(X_test.min(),X_test.max())

0.0 1.0000000000000002
-0.26669821984794406 3.110700227058437


In [97]:
from sklearn.ensemble import RandomForestClassifier
clf =RandomForestClassifier()
print(clf.get_params())
param_grid = {
    'n_estimators': [10,50,100,150,200]
     }
# Instantiate a 10-fold CV grid search object 'grid_dt'
grid_dt= GridSearchCV(estimator=clf,
                       param_grid= param_grid,
                       scoring='accuracy',
                       cv=10,
                       n_jobs=8)

# Fit 'grid_dt' to the training data
grid_dt.fit(X_train, y_train)

# Extract best hyperparameters from 'grid_dt' and print them
best_hyperparams = grid_dt.best_params_
best_CV_score = grid_dt.best_score_
print(best_hyperparams,best_CV_score)

{'bootstrap': True, 'ccp_alpha': 0.0, 'class_weight': None, 'criterion': 'gini', 'max_depth': None, 'max_features': 'auto', 'max_leaf_nodes': None, 'max_samples': None, 'min_impurity_decrease': 0.0, 'min_impurity_split': None, 'min_samples_leaf': 1, 'min_samples_split': 2, 'min_weight_fraction_leaf': 0.0, 'n_estimators': 100, 'n_jobs': None, 'oob_score': False, 'random_state': None, 'verbose': 0, 'warm_start': False}
{'n_estimators': 100} 0.8463583556747096


In [98]:
#
clf = RandomForestClassifier(n_estimators=200).fit(X_train, y_train)
#
#training 
y_pred_train=clf.predict(X_train)
#------------------------------------------#
#testing
y_pred_test=clf.predict(X_test)
#
cm = confusion_matrix(y_test,y_pred_test)
f1 = f1_score(y_test,y_pred_test)
acc= accuracy_score(y_test,y_pred_test)
cm,f1,acc

(array([[615,  46],
        [ 97, 173]], dtype=int64),
 0.7075664621676891,
 0.8464017185821697)

In [91]:
#
clf = SVC(kernel= 'linear').fit(X_train, y_train)
#
#training 
y_pred_train=clf.predict(X_train)
#------------------------------------------#
#testing
y_pred_test=clf.predict(X_test)
#
cm = confusion_matrix(y_test,y_pred_test)
f1 = f1_score(y_test,y_pred_test)
acc= accuracy_score(y_test,y_pred_test)
cm,f1,acc

(array([[656,   5],
        [254,  16]], dtype=int64),
 0.10996563573883161,
 0.7218045112781954)